In [1]:
## Week 6: Data Retrieval and Processing

In [2]:
!pip install web3

In [2]:
#Establishment of Web 3 connection to the Ganache RPC URL
from web3 import Web3


# Connect to local blockchain
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))


# Verify connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")

✅ Connected to Ganache successfully!


In [3]:
# Replace with actual contract address from Remix
contract_address = "0xF722Ecca72f1F3C33FBd7c1Bf8FCd29BA4BcE883"


# Paste the ABI from Remix
abi = [
    {
        'inputs': [],
        'stateMutability': 'nonpayable',
        'type': 'constructor'
    },
    {
        'anonymous': False,
        'inputs': [
            {'indexed': False, 'internalType': 'uint256', 'name': 'timestamp', 'type': 'uint256'},
            {'indexed': False, 'internalType': 'string', 'name': 'sensorId', 'type': 'string'},
            {'indexed': False, 'internalType': 'string', 'name': 'dataType', 'type': 'string'},
            {'indexed': False, 'internalType': 'string', 'name': 'dataValue', 'type': 'string'}
        ],
        'name': 'DataRetrieved',
        'type': 'event'
    },
    {
        'anonymous': False,
        'inputs': [
            {'indexed': False, 'internalType': 'uint256', 'name': 'timestamp', 'type': 'uint256'},
            {'indexed': False, 'internalType': 'string', 'name': 'sensorId', 'type': 'string'},
            {'indexed': False, 'internalType': 'string', 'name': 'dataType', 'type': 'string'},
            {'indexed': False, 'internalType': 'string', 'name': 'dataValue', 'type': 'string'}
        ],
        'name': 'DataStored',
        'type': 'event'
    },
    {
        'inputs': [
            {'internalType': 'string', 'name': '_sensorId', 'type': 'string'},
            {'internalType': 'string', 'name': '_dataType', 'type': 'string'},
            {'internalType': 'string', 'name': '_dataValue', 'type': 'string'}
        ],
        'name': 'storeData',
        'outputs': [],
        'stateMutability': 'nonpayable',
        'type': 'function'
    },
    {
        'inputs': [
            {'internalType': 'uint256', 'name': '', 'type': 'uint256'}
        ],
        'name': 'dataRecords',
        'outputs': [
            {'internalType': 'uint256', 'name': 'timestamp', 'type': 'uint256'},
            {'internalType': 'string', 'name': 'sensorId', 'type': 'string'},
            {'internalType': 'string', 'name': 'dataType', 'type': 'string'},
            {'internalType': 'string', 'name': 'dataValue', 'type': 'string'}
        ],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [
            {'internalType': 'uint256', 'name': 'index', 'type': 'uint256'}
        ],
        'name': 'getRecord',
        'outputs': [
            {'internalType': 'uint256', 'name': '', 'type': 'uint256'},
            {'internalType': 'string', 'name': '', 'type': 'string'},
            {'internalType': 'string', 'name': '', 'type': 'string'},
            {'internalType': 'string', 'name': '', 'type': 'string'}
        ],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [],
        'name': 'getTotalRecords',
        'outputs': [
            {'internalType': 'uint256', 'name': '', 'type': 'uint256'}
        ],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [],
        'name': 'MAX_ENTRIES',
        'outputs': [
            {'internalType': 'uint256', 'name': '', 'type': 'uint256'}
        ],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [],
        'name': 'owner',
        'outputs': [
            {'internalType': 'address', 'name': '', 'type': 'address'}
        ],
        'stateMutability': 'view',
        'type': 'function'
    }
]
  # Replace with your contract ABI


# Load the smart contract
contract = web3.eth.contract(address=contract_address, abi=abi)


# Set the default sender address (first account from Ganache)
web3.eth.default_account = web3.eth.accounts[0]


print(f"✅ Connected to Smart Contract at {contract_address}")

✅ Connected to Smart Contract at 0xF722Ecca72f1F3C33FBd7c1Bf8FCd29BA4BcE883


In [4]:
## Verify if total records is accurate which is 700:
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

Total IoT records stored: 700


In [5]:
## Fetch all stored IoT data and structure it in a DataFrame
import pandas as pd


# Retrieve all IoT records
data = []
for i in range(total_records):
    record = contract.functions.getRecord(i).call()
    data.append({
        "timestamp": record[0],
        "sensor_id": record[1],
        "data_type": record[2],
        "data_value": record[3]
    })


# Convert to a DataFrame
df = pd.DataFrame(data)


# Convert timestamp to readable format
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")


# Display first few records
print(df.head())

            timestamp sensor_id              data_type   data_value
0 2025-06-03 18:41:59   SENS762                co2_ppm  433.0438194
1 2025-06-03 18:42:00   SENS762              pm25_ugm3  35.41177588
2 2025-06-03 18:42:01   SENS762          temperature_c         30.7
3 2025-06-03 18:42:02   SENS762       humidity_percent         64.6
4 2025-06-03 18:42:03   SENS762  soil_moisture_percent         28.8


In [6]:
## Data Cleaning
import numpy as np


# Extract numeric values from 'data_value' where applicable
df["numeric_value"] = df["data_value"].str.extract(r'(\d+\.?\d*)').astype(float)


# Handle missing values (if any)
df.fillna(0, inplace=True)

# Round numeric values to two decimal places
df["numeric_value"] = df["numeric_value"].round(2)

# Display cleaned data
print(df.head())

            timestamp sensor_id              data_type   data_value  \
0 2025-06-03 18:41:59   SENS762                co2_ppm  433.0438194   
1 2025-06-03 18:42:00   SENS762              pm25_ugm3  35.41177588   
2 2025-06-03 18:42:01   SENS762          temperature_c         30.7   
3 2025-06-03 18:42:02   SENS762       humidity_percent         64.6   
4 2025-06-03 18:42:03   SENS762  soil_moisture_percent         28.8   

   numeric_value  
0         433.04  
1          35.41  
2          30.70  
3          64.60  
4          28.80  


In [7]:
# Save cleaned IoT data to a CSV file
df.to_csv("cleaned_iot_data.csv", index=False)


print("✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv")

✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv
